In [26]:
import sys
sys.path.append("../model")
from MLP_classifier import MultiClassClassifier
sys.path.append("../tools")
import open_clip
import torch
import os
from PIL import Image
from dataset import TaskA
from pathlib import Path
import pandas as pd

### ATTENTION : 0 -> image FAKE et 1 -> image REELLE (voir constant.py REAL_LABEL et FAKE_LABEL) -> c'est l'opposé des labels utilisés dans le challenge !!! Il faut inverser les prédictions pour obtenir le résultat souhaité (c'est ce que j'ai fait dans la dernière cellule)

In [2]:
device = "cuda:0"

In [16]:
model = MultiClassClassifier(n_classes=2).to(device)
model.load_state_dict(torch.load("../model/checkpoints/binary_train_real_fake_2k_fine_tune_meta_test.pt"))
model.eval()

MultiClassClassifier(
  (fc1): Linear(in_features=768, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
  (act): ReLU()
)

### Extracting clip features

In [21]:
data = TaskA(load_from_disk=False,
             path="/data3/TEST/A/", # replace with new path
             device=device)

In [36]:
path_meta = "/data3/TEST/A/"

path = Path(path_meta)
im_path = sorted(list(path.rglob('*.jpg'))+list(path.rglob('*.jpeg'))+list(path.rglob('*.png')))
im_names = [p.name for p in im_path]
im_dir = [str(p.parents[0]) for p in im_path]

df = pd.DataFrame(columns=['dir_name','image_name','pred'])
df['image_name'] = im_names
df['dir_name'] = im_dir

df.head()

,dir_name,image_name,pred
0,/data3/TEST/A,A_005fbfn6.png,NaN
1,/data3/TEST/A,A_0060ug9j.png,NaN
2,/data3/TEST/A,A_00el4hwr.png,NaN
3,/data3/TEST/A,A_00h6ucsm.png,NaN
4,/data3/TEST/A,A_00hzao5p.png,NaN


### Predictions

In [45]:
pred_list = []
for index, row in df.iterrows():
    with torch.no_grad():
        # pred = 1 - model.predict... -> label 0 = real img and label 1 = fake image
        pred = 1 - model.predict_binary(data.features[row['image_name']].to(device),device,True) 
    pred_list.append(pred.cpu().item())

df['pred']= pred_list
df.head(5)
    

,dir_name,image_name,pred
0,/data3/TEST/A,A_005fbfn6.png,1
1,/data3/TEST/A,A_0060ug9j.png,1
2,/data3/TEST/A,A_00el4hwr.png,1
3,/data3/TEST/A,A_00h6ucsm.png,1
4,/data3/TEST/A,A_00hzao5p.png,0
